In [39]:
import pandas as pd
import zipcodes

df_raw = pd.read_csv('./No Kill Colorado - Facilities Lat Long.csv')

In [41]:
df_raw

,facility_name,zip_code,State,Country,FID,x,y
0,"2 Blondes All Breed Rescue, Inc",80232.0,Colorado,United States,71,-105.087784,39.692628
1,"2Nd Chance Vizsla Rescue, Inc",80524.0,Colorado,United States,496,-105.049207,40.612550
2,4 Paws 4 Life Rescue,80135.0,Colorado,United States,202,-105.001236,39.346209
3,5280 Reptile Room North,80233.0,Colorado,United States,519,-104.976227,39.913286
4,5280 Reptile Room West,80033.0,Colorado,United States,458,-105.110438,39.773954
...,...,...,...,...,...,...,...
601,Wolfwood,80814.0,Colorado,United States,374,-105.175833,38.933470
602,Woof And Willow Company,80209.0,Colorado,United States,218,-104.965148,39.711910
603,World Of Birds,80104.0,Colorado,United States,460,-105.083084,39.748101
604,"X And B Pets, Inc",80220.0,Colorado,United States,555,-104.915035,39.732680


In [42]:
# Check columns and create facility-county mapping using zip_code
print("Available columns in df_raw:")
print(df_raw.columns.tolist())
print(f"\nDataset shape: {df_raw.shape}")

Available columns in df_raw:
['facility_name', 'zip_code', 'State', 'Country', 'FID', 'x', 'y']

Dataset shape: (606, 7)


In [43]:
# Create facility-county mapping using facility_name and zip_code columns
print("Creating facility-county mapping using facility_name and zip_code columns...")

# Use existing zipcode-county mapping as lookup
zipcode_to_county_lookup = {}
for _, row in mapping_df.iterrows():
    zipcode_to_county_lookup[row['original_zipcode']] = {
        'county': row['county'],
        'city': row['city'],
        'formatted_zipcode': row['formatted_zipcode']
    }

# Create facility-county mapping
facility_county_data = []
unmapped_facilities = []

# Process each row in the dataset
for _, row in df_raw.iterrows():
    facility_name = row['facility_name']
    zip_code = row['zip_code']
    
    if pd.notna(facility_name) and pd.notna(zip_code):
        try:
            # Convert zip_code to int for matching
            zip_int = int(float(zip_code))
            
            if zip_int in zipcode_to_county_lookup:
                county_info = zipcode_to_county_lookup[zip_int]
                facility_county_data.append({
                    'facility_name': facility_name,
                    'zip_code': zip_int,
                    'formatted_zipcode': county_info['formatted_zipcode'],
                    'city': county_info['city'],
                    'county': county_info['county']
                })
            else:
                unmapped_facilities.append({
                    'facility_name': facility_name,
                    'zip_code': zip_int
                })
        except (ValueError, TypeError):
            print(f"Error processing zip_code: {zip_code} for facility: {facility_name}")
            continue

# Create DataFrame and remove duplicates
facility_county_df = pd.DataFrame(facility_county_data)
if not facility_county_df.empty:
    facility_county_df = facility_county_df.drop_duplicates()

unmapped_facilities_df = pd.DataFrame(unmapped_facilities)
if not unmapped_facilities_df.empty:
    unmapped_facilities_df = unmapped_facilities_df.drop_duplicates()

# Save to CSV
facility_county_df.to_csv('facility_name_county_mapping.csv', index=False)

# Print statistics
total_facilities = len(df_raw['facility_name'].dropna().unique())
mapped_facilities = len(facility_county_df['facility_name'].unique()) if not facility_county_df.empty else 0
unmapped_count = len(unmapped_facilities_df['facility_name'].unique()) if not unmapped_facilities_df.empty else 0

print(f"\nFacility-County Mapping Results:")
print(f"Total unique facilities: {total_facilities}")
print(f"Facilities mapped to counties: {mapped_facilities}")
print(f"Facilities not mapped: {unmapped_count}")
print(f"Mapping success rate: {(mapped_facilities/total_facilities)*100:.1f}%")

print(f"\nMapping saved to 'facility_name_county_mapping.csv'")

# Show sample of mappings
if not facility_county_df.empty:
    print(f"\nSample facility-county mappings:")
    print(facility_county_df.head(10))

# Show county distribution
if not facility_county_df.empty:
    print(f"\nTop counties by facility count:")
    county_counts = facility_county_df['county'].value_counts().head(10)
    print(county_counts)

# Show unmapped facilities if any
if not unmapped_facilities_df.empty and len(unmapped_facilities_df) > 0:
    print(f"\nSample unmapped facilities:")
    print(unmapped_facilities_df.head(5))

Creating facility-county mapping using facility_name and zip_code columns...

Facility-County Mapping Results:
Total unique facilities: 597
Facilities mapped to counties: 553
Facilities not mapped: 29
Mapping success rate: 92.6%

Mapping saved to 'facility_name_county_mapping.csv'

Sample facility-county mappings:
                                     facility_name  zip_code  \
0                  2 Blondes All Breed Rescue, Inc     80232   
1                    2Nd Chance Vizsla Rescue, Inc     80524   
2                             4 Paws 4 Life Rescue     80135   
3                          5280 Reptile Room North     80233   
4                           5280 Reptile Room West     80033   
5                              7 Paws Rescue Ranch     81621   
6                                   9 Lives Rescue     80907   
7                   A Cat Rescue Out In The Sticks     80727   
8                          A Friend Of Jack Rescue     80223   
9  Adams County Animal Shelter And Adoption 

In [44]:
# Summary of the facility-county mapping results
print("=== FACILITY-COUNTY MAPPING SUMMARY ===")
print(f"CSV file created: 'facility_name_county_mapping.csv'")
print(f"Total records in CSV: {len(facility_county_df)}")
print(f"Unique facilities mapped: {len(facility_county_df['facility_name'].unique())}")
print(f"Counties covered: {len(facility_county_df['county'].unique())}")

print("\nTop 5 counties by facility count:")
top_counties = facility_county_df['county'].value_counts().head(5)
for county, count in top_counties.items():
    print(f"  {county}: {count} facilities")

print(f"\nFile successfully saved with columns:")
print(facility_county_df.columns.tolist())

=== FACILITY-COUNTY MAPPING SUMMARY ===
CSV file created: 'facility_name_county_mapping.csv'
Total records in CSV: 553
Unique facilities mapped: 553
Counties covered: 53

Top 5 counties by facility count:
  Denver: 67 facilities
  Jefferson: 58 facilities
  Douglas: 57 facilities
  Arapahoe: 51 facilities
  El Paso: 48 facilities

File successfully saved with columns:
['facility_name', 'zip_code', 'formatted_zipcode', 'city', 'county']
